# 나이브베이즈_텍스트 분석 수행

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

In [1]:
from konlpy.tag import Mecab

mecab = Mecab()
print(mecab.morphs(u'영등포구청역에 있는 맛집 좀 알려주세요.'), end='\n\n')
print(mecab.nouns(u'영등포구청역에 있는 맛집 좀 알려주세요.'), end='\n\n')
print(mecab.pos(u'영등포구청역에 있는 맛집 좀 알려주세요.'))

['영등포구청역', '에', '있', '는', '맛집', '좀', '알려', '주', '세요', '.']

['영등포구청역', '맛집']

[('영등포구청역', 'NNP'), ('에', 'JKB'), ('있', 'VV'), ('는', 'ETM'), ('맛집', 'NNG'), ('좀', 'MAG'), ('알려', 'VV+EC'), ('주', 'VX'), ('세요', 'EP+EF'), ('.', 'SF')]


## train 데이터 불러오기 및 확인

In [3]:
train_df = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/ratings_train.txt', sep='\t')
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [5]:
train_df = train_df.dropna()

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [7]:
train_df['label'].value_counts(normalize=True)

label
0    0.50115
1    0.49885
Name: proportion, dtype: float64

## test 데이터 불러오기 및 확인

In [8]:
test_df = pd.read_csv('./data/ratings_test.txt', sep='\t')
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [9]:
test_df['label'].value_counts()

label
1    25173
0    24827
Name: count, dtype: int64

In [10]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [11]:
test_df = test_df.dropna()
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


## 나이브베이즈 모델 적용 위한 전처리

In [12]:
# 특수문자 처리가 필요
train_df['document']

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

### 정규표현식을 이용한 텍스트 정리

In [13]:
import re

In [14]:
def text_clean(x):
    pattern = r'[가-힣0-9a-zA-Z]+'
    matches = re.findall(pattern, x)
    matches = ' '.join(matches)
    return matches

In [15]:
# 정규표현식을 통한 텍스트 변환
# 띄워쓰기 잘 분리되지 않은 것은 형태소 변환 시에 나누어지기 때문에 크게 상관 없음
train_df['document'].apply(text_clean)

0                                         아 더빙 진짜 짜증나네요 목소리
1                              흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                                교도소 이야기구먼 솔직히 재미는 없다 평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                      인간이 문제지 소는 뭔죄인가
149996                                           평점이 너무 낮아서
149997                        이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
149998                          청춘 영화의 최고봉 방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [16]:
train_df['document'] = train_df['document'].apply(text_clean)

/tmp/ipykernel_1133/1142159049.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['document'] = train_df['document'].apply(text_clean)


In [17]:
train_df['document']

0                                         아 더빙 진짜 짜증나네요 목소리
1                              흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                                교도소 이야기구먼 솔직히 재미는 없다 평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                      인간이 문제지 소는 뭔죄인가
149996                                           평점이 너무 낮아서
149997                        이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
149998                          청춘 영화의 최고봉 방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [18]:
X = train_df['document']
y = train_df['label']

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=10)

- document에 있는 문자를 숫자로 변환
- 문자의 벡터화 : CountVectorizer()


In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
# 문자의 벡터화
cv = CountVectorizer() # 토크나이저가 공백을 기준으로 토큰화
cv.fit(X_train)
X_train = cv.transform(X_train)
X_valid = cv.transform(X_valid)

- 독립변수가 문자로 되어 있는 데이터의 경우 MultimomialNB
- 독립변수가 수치형으로 이루어진 경우 : GaussianNB

In [23]:
from sklearn.naive_bayes import MultinomialNB

In [24]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
pred = mnb.predict(X_valid)

print(classification_report(y_valid, pred))

              precision    recall  f1-score   support

           0       0.81      0.84      0.82     22638
           1       0.83      0.80      0.82     22361

    accuracy                           0.82     44999
   macro avg       0.82      0.82      0.82     44999
weighted avg       0.82      0.82      0.82     44999



### test 데이터 불러와서 검증

In [25]:
test_df['document']

0                                                      굳 ㅋ
1                                     GDNTOPCLASSINTHECLUB
2                   뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                         지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4        3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
                               ...                        
49995            오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996         의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997                   그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
49998       절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네
49999                                           마무리는 또 왜이래
Name: document, Length: 49997, dtype: object

In [26]:
X_test = test_df['document']
y_test = test_df['label']

In [27]:
X_test = cv.transform(X_test)

In [28]:
test_pred = mnb.predict(X_test)
print(classification_report(y_test, test_pred))

              precision    recall  f1-score   support

           0       0.80      0.84      0.82     24826
           1       0.83      0.80      0.81     25171

    accuracy                           0.82     49997
   macro avg       0.82      0.82      0.82     49997
weighted avg       0.82      0.82      0.82     49997



# Konlpy의 Mecab을 이용한 형태소 분리

In [30]:
from konlpy.tag import Mecab
mecab = Mecab()

In [29]:
def tokenizer(text):
    tokens = mecab.morphs(text)
    return tokens

# CountVectorize, TfidfVectorizer

## 1) CountVectorizer: 단어 빈도(횟수) 기반 벡터화
- 작동 방식: CountVectorizer는 각 문서에서 등장한 단어의 **횟수(빈도)**를 기준으로 벡터 생성 
- 단순히 각 문서에서 특정 단어가 몇 번 등장했는지를 세어 벡터화

- 특징
    - 각 단어의 빈도가 높을수록 해당 단어의 중요성이 더 크다고 가정
    - 빈도가 높다는 것만을 고려하기 때문에, 모든 문서에서 자주 등장하는 단어도 중요한 단어로 처리 가능
    
- 예시
    예를 들어, 두 개의 문서가 있다면:<br>
    문서 1: "고양이가 나무 위에 있다."<br>
    문서 2: "나무 아래에 고양이가 있다."<br>
    이 두 문서를 CountVectorizer로 변환하면 단어 빈도가 포함된 벡터가 생성됩니다:<br>
    ['고양이': 2, '나무': 2, '위에': 1, '아래에': 1, '있다': 2]<br>
    
    
### 2) TfidfVectorizer: TF-IDF (Term Frequency-Inverse Document Frequency) 기반 벡터화
- 작동 방식: TfidfVectorizer는 **단어의 빈도뿐만 아니라, 단어의 중요도**를 계산
- TF-IDF 값을 사용하여 문서 간 차별성 강조
    - TF (Term Frequency): 단어가 문서에서 얼마나 자주 등장했는지를 나타냄
    - IDF (Inverse Document Frequency): 단어가 다른 문서에 얼마나 자주 등장하지 않았는지를 나타냄(자주 등장하지 않는 단어는 더 중요한 단어로 간주)
- 특징
    - TF-IDF는 문서 전체에서 자주 등장하는 흔한 단어들(예: "그리고", "이다" 등)의 중요도를 낮추고, 문서에서만 중요한 단어들의 중요도를 높dla
    - 단순히 빈도가 높은 단어보다 특정 문서에서 더 특징적인 단어에 더 높은 가중치를 부여
- 예시
    위의 문서 1과 문서 2에 대해 TfidfVectorizer로 변환하면,<br>
    공통 단어들(예: "있다", "고양이")의 중요도는 낮아지고,<br>
    차별적인 단어(예: "위에", "아래에")의 중요도는 상대적으로 높아집니다.

In [32]:
# countvectorize
from sklearn.feature_extraction.text import CountVectorizer

In [33]:
doc = ['고양이가 나무 위에 있다.', '나무 아래에 고양이가 있다.']

In [34]:
c_vec = CountVectorizer()
X = c_vec.fit_transform(doc)
print(c_vec.get_feature_names_out()) # 벡터화된 단어 목록
print(X.toarray()) # 단어 빈도 벡터


['고양이가' '나무' '아래에' '위에' '있다']
[[1 1 0 1 1]
 [1 1 1 0 1]]


In [35]:
# tfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
tfidf = TfidfVectorizer()
X2 = tfidf.fit_transform(doc)
print(tfidf.get_feature_names_out()) # 벡터화된 단어 목록
print(X2.toarray()) # 단어 빈도 벡터

['고양이가' '나무' '아래에' '위에' '있다']
[[0.44832087 0.44832087 0.         0.63009934 0.44832087]
 [0.44832087 0.44832087 0.63009934 0.         0.44832087]]


## CountVectorizer와 TfidfVectorizer 비교

| **특성**             | **CountVectorizer**                                     | **TfidfVectorizer**                                               |
|----------------------|--------------------------------------------------------|-------------------------------------------------------------------|
| **기반**             | 단어의 단순 빈도                                        | 단어 빈도 + 문서 내에서의 상대적 중요도(TF-IDF)                     |
| **단어 빈도 계산**    | 문서에서 등장한 단어의 단순한 등장 횟수를 셈            | 단어의 등장 횟수(TF)와 해당 단어가 문서들에서 얼마나 자주 등장하지 않았는지를 함께 고려(IDF) |
| **빈번한 단어 처리**  | 문서에서 자주 등장하는 단어일수록 높은 가중치를 부여    | 문서에서 흔한 단어는 가중치를 낮추고, 드문 단어는 높은 가중치를 부여   |
| **주요 용도**        | 단순한 단어 빈도 기반 분석이 필요할 때 사용             | 문서 간 차별적인 단어를 구별할 때 유용                                |
| **계산 비용**         | 상대적으로 적음                                         | 상대적으로 더 복잡하고 계산 비용이 높음                                |


- tokenizer : 형태소를 분석하여 단어를 나누어 주는 역할, 기본은 공백을 기준으로 나눔
    - 한국어는 조사가 있기 때문에 공백이 아닌 형태소 분석을 통해서 나눠야 함
    - konlpy의 형태소 분석기를 이용해 형태소를 나누고 분석
- ngram_range(1,2)
    - 단어를 벡터화 하는 경우 단어의 범위를 지정 (1,2)sms 1-gram, 2-gram
    - '이 영화는 정말 좋다'
    - 1-gram : ['이', '영화는', '정말', '좋다']
    - 2-gram : ['이 영화는', '영화는 정말', '정말 좋다']
- min_df = 3
    - 단어가 등장하는 최소 문서 수를 설정하는 파라미터
- max_df = 0.9
    - 단어가 전체 문서의 90% 이하에서 등장하는 경우에만 벡터화에 포함
    

In [37]:
X = train_df['document']
y = train_df['label']

In [38]:
tf_X_train, tf_X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
# tokenizer를 konlpy의 mecab 사용
tfidf = TfidfVectorizer(tokenizer=tokenizer, ngram_range=(1,2), min_df=4, max_df=0.9)
tf_X_train = tfidf.fit_transform(tf_X_train)
tf_X_valid = tfidf.transform(tf_X_valid)

/home/fintech/miniforge3/envs/fintech/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [ ]:
%%time

mnb = MultinomialNB()
mnb.fit(tf_X_train, y_train)
pred = mnb.predict(tf_X_valid)
print(classification_report(y_valid, pred))

In [ ]:
X_test = test_df['document']
y_test = test_df['label']

In [ ]:
tf_X_test = tfidf.transform(X_test)

In [ ]:
test_pred = mnb.predict(tf_X_test)
print(classification_report(y_test, test_pred))

# Random_forest와 비교

In [46]:
from sklearn.ensemble import RandomForestClassifier

In [51]:
# 시간이 너무 오래 걸려서 넘김
rfc = RandomForestClassifier(n_jobs=8, random_state=10)
rfc.fit(tf_X_train, y_train)
pred = rfc.predict(tf_X_valid)
print(classification_report(y_valid, pred))

In [52]:
# 시간이 너무 오래 걸려서 넘김
rfc_test_pred = rfc.predict(tf_X_test)
print(classification_report(y_test, rfc_test_pred))